In [1]:
from __future__ import print_function
from unicodecsv import DictReader
import pandas as pd
from paths import DATASETS_DIR, DUMPS_DIR
from pprint import pprint
from data import generate_dump
import os
from collections import OrderedDict
# import mysql.connector
from sqlalchemy import create_engine
# import pyodbc
from helpers import freq_iso_to_pandas
from pydatajson.helpers import parse_repeating_time_interval_to_days
from generate_dumps import save_to_dta, generate_series_summary
import numpy as np

In [2]:
from pympler import asizeof

In [3]:
dump_path = os.path.join(DUMPS_DIR, "series-tiempo.{}")

In [12]:
df = pd.read_excel(os.path.join(DUMPS_DIR, "series-tiempo-metadatos.xlsx"))

In [17]:
df.groupby("serie_actualizada").count()

,catalog_id,dataset_id,distribucion_id,serie_id,indice_tiempo_frecuencia,serie_titulo,serie_unidades,serie_descripcion,distribucion_titulo,distribucion_descripcion,...,dataset_descripcion,serie_indice_inicio,serie_indice_final,serie_valores_cant,serie_dias_no_cubiertos,serie_valor_ultimo,serie_valor_anterior,serie_valor_anterior_anio,serie_var_pct_anterior,serie_var_pct_anterior_anio
serie_actualizada,,,,,,,,,,,,,,,,,,,,,
False,3886,3886,3886,3886,3886,3886,3885,3886,3886,3886,...,3886,3886,3886,3886,3886,3710,3675,3539,3556,3406
True,4388,4388,4388,4388,4388,4388,4388,4388,4388,4388,...,4388,4388,4388,4388,4388,4086,4213,4124,4053,3880


In [15]:
print(df.serie_indice_inicio.min())
print(df.serie_indice_final.max())

1935-01-01 00:00:00
2018-07-01 00:00:00


## Genero dataframe con el dump

In [3]:
%time df = generate_dump(merged=True)

0 datasets para agregar del catalogo modernizacion
238 datasets para agregar del catalogo sspm
distribution 34.2 no tiene `field`
0 datasets para agregar del catalogo sspmi
CPU times: user 1min 35s, sys: 6.23 s, total: 1min 42s
Wall time: 1min 43s


In [12]:
if "indice_tiempo" in df.columns:
    print("aa")

aa


In [11]:
df.indice_tiempo.dt.strftime("%Y-%m-%d")

0          1993-01-01
1          1994-01-01
2          1995-01-01
3          1996-01-01
4          1997-01-01
5          1998-01-01
6          1999-01-01
7          2000-01-01
8          2001-01-01
9          2002-01-01
10         2003-01-01
11         2004-01-01
12         2005-01-01
13         2006-01-01
14         2007-01-01
15         2008-01-01
16         2009-01-01
17         2010-01-01
18         2011-01-01
19         2012-01-01
20         1993-01-01
21         1994-01-01
22         1995-01-01
23         1996-01-01
24         1997-01-01
25         1998-01-01
26         1999-01-01
27         2000-01-01
28         2001-01-01
29         2002-01-01
              ...    
1125908    2011-07-01
1125909    2011-08-01
1125910    2011-09-01
1125911    2011-10-01
1125912    2011-11-01
1125913    2011-12-01
1125914    2012-01-01
1125915    2012-02-01
1125916    2012-03-01
1125917    2012-04-01
1125918    2012-05-01
1125919    2012-06-01
1125920    2012-07-01
1125921    2012-08-01
1125922   

In [6]:
for col in df.columns:
    print(col, df[col].dtype)

catalog_id object
dataset_id object
distribution_id object
serie_id object
indice_tiempo datetime64[ns]
indice_tiempo_frecuencia object
valor float64
serie_titulo object
serie_unidades object
serie_descripcion object
distribution_titulo object
distribution_descripcion object
dataset_responsable object
dataset_fuente object
dataset_titulo object
dataset_descripcion object


In [ ]:
df_stata = df

In [ ]:
for col in df_stata.columns:

    # limita el largo de los campos de texto
    if df_stata[col].dtype.name == "object":
        df_stata[col] = df_stata[col].astype(unicode).str[:str_limit]

    # elimina los valores infinitos de los tipos decimales
    elif "float" in df_stata[col].dtype.name:
        df_stata[col] = df_stata[col].apply(
            lambda x: np.nan if np.isinf(x) else x)

# df_stata.to_stata(path, write_index=False)

In [5]:
(asizeof.asizeof(df_values) + asizeof.asizeof(df_distrib_metadata)) / 1000000000.0

2.240693368

In [5]:
asizeof.asizeof(df) / 1000000000.0

5.208717272

In [11]:
len(df[df["indice_tiempo_frecuencia"] == "R/P1D"])

409845

In [7]:
df.groupby("indice_tiempo_frecuencia").count()

,catalog_id,dataset_id,distribucion_id,serie_id,indice_tiempo,valor,serie_titulo,serie_unidades,serie_descripcion,distribucion_titulo,distribucion_descripcion,dataset_responsable,dataset_fuente,dataset_titulo,dataset_descripcion
indice_tiempo_frecuencia,,,,,,,,,,,,,,,
R/P1D,409845,409845,409845,409845,409845,331095,409845,409845,409845,409845,409845,409845,409845,409845,409845
R/P1M,470112,470112,470112,470112,470112,466610,470112,470112,470112,470112,470112,470112,470112,470112,470112
R/P1Y,53306,53306,53306,53306,53306,53112,53306,53299,53306,53306,53306,53306,53306,53306,53306
R/P3M,192798,192798,192798,192798,192798,191593,192798,192798,192798,192798,192798,192798,192798,192798,192798
R/P6M,1196,1196,1196,1196,1196,1196,1196,1196,1196,1196,1196,1196,1196,1196,1196


In [36]:
def remove_leading_nans(df, serie_id):
    df_serie = df[df.serie_id == serie_id]
    first_positive = df_serie[df_serie.valor.notnull()].index[0]
    df[df.serie_id == serie_id] = df_serie.loc[first_positive:]

In [37]:
%%time
for serie_id in df_sorted.serie_id.unique():
    remove_leading_nans(df_sorted, serie_id)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
len(df_sorted)

In [29]:
df[df.serie_id == "149.1_TL_INDIIOS_OCTU_0_21"] = df[df.serie_id == "149.1_TL_INDIIOS_OCTU_0_21"].loc[first:]

In [23]:
first = df[df.serie_id == "149.1_TL_INDIIOS_OCTU_0_21"][df.valor.notnull()].index[0]

/Users/abenassi/anaconda/envs/series-tiempo/lib/python2.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


## Genero resumen de series

In [14]:
drop_cols = ["indice_tiempo", "valor"]
index_cols = ["catalog_id", "dataset_id", "distribution_id", "serie_id"]
series_cols = df.columns.drop(drop_cols)
series_group = df.groupby(index_cols)

In [15]:
df_series = df.drop(drop_cols, axis=1).drop_duplicates().set_index(index_cols)

In [16]:
def _is_series_updated(row):
    index_freq = row["indice_tiempo_frecuencia"]
    period_days = parse_repeating_time_interval_to_days(index_freq)
    periods_tolerance = {
        "R/P1Y": 2,
        "R/P6M": 2,
        "R/P3M": 2,
        "R/P1M": 3,
        "R/P1D": 14
    }
    days_tolerance = periods_tolerance.get(index_freq, 2) * period_days
    return row["serie_dias_no_cubiertos"] < days_tolerance    

In [17]:
def _get_serie_valor_anterior_anio(series_dataframe):
    series = pd.Series(list(series_dataframe.valor), list(
        series_dataframe.indice_tiempo)).sort_index()
    return series.get(series.index[-1] - pd.DateOffset(years=1))

In [18]:
# CALCULA INDICADORES resumen sobre las series
# rango temporal de la serie
df_series["serie_indice_inicio"] = series_group["indice_tiempo"].min()
df_series["serie_indice_final"] = series_group["indice_tiempo"].max()
df_series["serie_valores_cant"] = series_group["indice_tiempo"].count()

# estado de actualización de los datos
# calcula días que pasaron por encima de período cubierto por datos
df_series["serie_dias_no_cubiertos"] = df_series.apply(
    lambda x: (pd.datetime.now() - pd.to_datetime(x[
        "serie_indice_final"]).to_period(
        freq_iso_to_pandas(x[
            "indice_tiempo_frecuencia"],
            how="end")).to_timestamp(how="end")).days,
    axis=1)
# si pasaron 2 períodos no cubiertos por datos, serie está desactualizada
df_series["serie_actualizada"] = df_series.apply(
    _is_series_updated, axis=1)

# valores representativos nominales
df_series["serie_valor_ultimo"] = series_group.apply(
    lambda x: x.loc[x.indice_tiempo.argmax(), "valor"])
df_series["serie_valor_anterior"] = series_group.apply(
    lambda x: pd.Series(list(x.valor), list(
        x.indice_tiempo)).sort_index()[-2]
)
df_series["serie_valor_anterior_anio"] = series_group.apply(
    _get_serie_valor_anterior_anio)

# valores representativos en variación porcentual
df_series["serie_var_pct_anterior"] = df_series[
    "serie_valor_ultimo"] / df_series["serie_valor_anterior"] - 1
df_series["serie_var_pct_anterior_anio"] = df_series[
    "serie_valor_ultimo"] / df_series["serie_valor_anterior_anio"] - 1

In [19]:
group_fuentes = df_series.groupby("dataset_fuente")

In [20]:
df_fuentes_valores = group_fuentes.sum()[["serie_valores_cant"]].rename(columns={"serie_valores_cant": "valores_cant"})

In [21]:
df_fuentes_series = group_fuentes.count()[["serie_titulo"]].rename(columns={"serie_titulo": "series_cant"})

In [22]:
group_fuentes.max()[["serie_indice_final"]].rename(columns={"serie_indice_final": "fecha_ultimo_valor"})

,fecha_ultimo_valor
dataset_fuente,
Administración Nacional de la Seguridad Social (ANSES),2017-08-01
"BCRA, MAE, Rofex y Bloomberg",2017-08-31
Banco Central de la República Argentina (BCRA),2017-09-26
Bancos Centrales,2017-09-14
Bloomberg,2017-09-14
Cámara Argentina de Fondos Comunes de Inversión,2017-08-01
Fondo Monetario Internacional (FMI),2017-06-01
Instituto Argentino de Mercado de Capitales (IAMC),2017-08-31
Instituto Nacional de Estadística y Censos (INDEC),2017-08-01


In [23]:
group_fuentes.min()[["serie_indice_inicio"]].rename(columns={"serie_indice_inicio": "fecha_primer_valor"})

,fecha_primer_valor
dataset_fuente,
Administración Nacional de la Seguridad Social (ANSES),1971-01-01
"BCRA, MAE, Rofex y Bloomberg",2002-03-05
Banco Central de la República Argentina (BCRA),1950-01-01
Bancos Centrales,2001-01-01
Bloomberg,1990-12-01
Cámara Argentina de Fondos Comunes de Inversión,2000-01-01
Fondo Monetario Internacional (FMI),1980-01-01
Instituto Argentino de Mercado de Capitales (IAMC),2004-10-04
Instituto Nacional de Estadística y Censos (INDEC),1980-01-01


In [24]:
pd.concat([df_fuentes_series, df_fuentes_valores], axis=1).sort_values("series_cant", ascending=False)

,series_cant,valores_cant
dataset_fuente,,
Instituto Nacional de Estadística y Censos (INDEC),5315,348844
Ministerio de Hacienda,1076,341052
Banco Central de la República Argentina (BCRA),597,60377
Fondo Monetario Internacional (FMI),381,79011
Bloomberg,259,148033
Ministerio de Agroindustria,151,13942
"Ministerio de Trabajo, Empleo y Seguridad Social",129,19144
Secretaría de Asuntos Económicos,115,2996
Cámara Argentina de Fondos Comunes de Inversión,33,2919


In [4]:
df_series = generate_series_summary(df, ["catalog_id", "dataset_id", "distribution_id", "serie_id"], ["indice_tiempo", "valor"])

In [6]:
for col in df_series.columns:
    print(col)

catalog_id
dataset_id
distribution_id
serie_id
indice_tiempo_frecuencia
serie_titulo
serie_unidades
serie_descripcion
distribution_titulo
distribution_descripcion
dataset_responsable
dataset_fuente
dataset_titulo
dataset_descripcion
serie_indice_inicio
serie_indice_final
serie_valores_cant
serie_dias_no_cubiertos
serie_actualizada
serie_valor_ultimo
serie_valor_anterior
serie_var_pct_anterior


In [39]:
engine = create_engine('sqlite:///{}'.format("prueba.db"), echo=True)

In [40]:
df_series.to_sql("series_tiempo", engine, index=False, if_exists="replace")

2017-09-27 21:02:42,298 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


09/27/2017 09:02:42 [INFO]: SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


2017-09-27 21:02:42,300 INFO sqlalchemy.engine.base.Engine ()


09/27/2017 09:02:42 [INFO]: ()


2017-09-27 21:02:42,303 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1


...

09/27/2017 09:02:42 [INFO]: INSERT INTO series_tiempo (catalog_id, dataset_id, distribution_id, serie_id, indice_tiempo_frecuencia, serie_titulo, serie_unidades, serie_descripcion, distribution_titulo, distribution_descripcion, dataset_responsable, dataset_fuente, dataset_titulo, dataset_descripcion, serie_indice_inicio, serie_indice_final, serie_valores_cant, serie_dias_no_cubiertos, serie_actualizada, serie_valor_ultimo, serie_valor_anterior, serie_valor_anterior_anio, serie_var_pct_anterior, serie_var_pct_anterior_anio) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)


2017-09-27 21:02:42,811 INFO sqlalchemy.engine.base.Engine ((u'sspm', u'1', u'1.1', u'1.1_DGCP_D_1993_A_27', u'R/P1Y', u'demanda_global_consumo_priv', u'Millones de pesos a precios de 1993', u'Consumo privado desestacionalizado, en millones de pesos de 1993 y valores trimestrales', u'Oferta y Demanda Global. Precios constantes desestacionalizados. Base 1993. Valores anuales.', u'Oferta y Demanda Globales por componente, a precios de comprador, en millones de pesos de 1993 y valores anuales desestacionalizados.', u'Ministerio de Hacienda. Secretar\xeda de Pol\xedtica Econ\xf3mica. Subsecretar\xeda de Programaci\xf3n Macroecon\xf3mica.', u'Instituto Nacional de Estad\xedstica y Censos (INDEC)', u'Oferta y Demanda Globales. Datos desestacionalizados. Base 1993', u'Componentes desestacionalizados de la oferta y demanda globales a precios de 1993.', '1993-01-01 00:00:00.000000', '2012-01-01 00:00:00.000000', 20, 1731, 0, 62643.7701039, 58881.473, 58881.473, 0.06389611047773891, 0.0638961104

09/27/2017 09:02:42 [INFO]: ((u'sspm', u'1', u'1.1', u'1.1_DGCP_D_1993_A_27', u'R/P1Y', u'demanda_global_consumo_priv', u'Millones de pesos a precios de 1993', u'Consumo privado desestacionalizado, en millones de pesos de 1993 y valores trimestrales', u'Oferta y Demanda Global. Precios constantes desestacionalizados. Base 1993. Valores anuales.', u'Oferta y Demanda Globales por componente, a precios de comprador, en millones de pesos de 1993 y valores anuales desestacionalizados.', u'Ministerio de Hacienda. Secretar\xeda de Pol\xedtica Econ\xf3mica. Subsecretar\xeda de Programaci\xf3n Macroecon\xf3mica.', u'Instituto Nacional de Estad\xedstica y Censos (INDEC)', u'Oferta y Demanda Globales. Datos desestacionalizados. Base 1993', u'Componentes desestacionalizados de la oferta y demanda globales a precios de 1993.', '1993-01-01 00:00:00.000000', '2012-01-01 00:00:00.000000', 20, 1731, 0, 62643.7701039, 58881.473, 58881.473, 0.06389611047773891, 0.06389611047773891), (u'sspm', u'1', u'1.1

2017-09-27 21:02:42,927 INFO sqlalchemy.engine.base.Engine COMMIT


09/27/2017 09:02:42 [INFO]: COMMIT


In [43]:
df_series.serie_valor_anterior_anio.astype()

Index([                 u'catalog_id',                  u'dataset_id',
                   u'distribution_id',                    u'serie_id',
          u'indice_tiempo_frecuencia',                u'serie_titulo',
                    u'serie_unidades',           u'serie_descripcion',
               u'distribution_titulo',    u'distribution_descripcion',
               u'dataset_responsable',              u'dataset_fuente',
                    u'dataset_titulo',         u'dataset_descripcion',
               u'serie_indice_inicio',          u'serie_indice_final',
                u'serie_valores_cant',     u'serie_dias_no_cubiertos',
                 u'serie_actualizada',          u'serie_valor_ultimo',
              u'serie_valor_anterior',   u'serie_valor_anterior_anio',
            u'serie_var_pct_anterior', u'serie_var_pct_anterior_anio'],
      dtype='object')

In [42]:
for index, col in enumerate(df_series.columns):
    print(index, col)

0 catalog_id
1 dataset_id
2 distribution_id
3 serie_id
4 indice_tiempo_frecuencia
5 serie_titulo
6 serie_unidades
7 serie_descripcion
8 distribution_titulo
9 distribution_descripcion
10 dataset_responsable
11 dataset_fuente
12 dataset_titulo
13 dataset_descripcion
14 serie_indice_inicio
15 serie_indice_final
16 serie_valores_cant
17 serie_dias_no_cubiertos
18 serie_actualizada
19 serie_valor_ultimo
20 serie_valor_anterior
21 serie_valor_anterior_anio
22 serie_var_pct_anterior
23 serie_var_pct_anterior_anio


In [20]:
df_series.to_excel("series-tiempo-resumen.xlsx", "resumen", index=True, index_label=True, merge_cells=False)

In [29]:
df_stata = df_series.copy()

In [35]:
for col in df_series.columns:
    
    # limita el largo de los campos de texto
    if df_stata[col].dtype.name == "object":
        df_stata[col] = df_stata[col].str[:244]
    
    # elimina los valores infinitos de los tipos decimales
    elif "float" in df_stata[col].dtype.name:
        df_stata[col] = df_stata[col].apply(lambda x: np.nan if np.isinf(x) else x)

In [36]:
df_stata.to_stata("series-tiempo-resumen.dta", write_index=False)

In [34]:
df_series.var_pct_anterior.max()

inf

In [ ]:
df_series.to_sql()

## Genero distribuciones de distintos formatos

In [ ]:
%%time
# CSV
path = dump_path.format("csv")
df.to_csv(path, encoding="utf-8", sep=",", index=False)
print("{}MB".format(os.path.getsize(path) / 1000000))

In [ ]:
%%time 
# EXCEL
path = dump_path.format("xlsx")
writer = pd.ExcelWriter(path, engine='xlsxwriter')
df.to_excel(writer, "data", merge_cells=False, encoding="utf-8", index=False)
writer.save()
print("{}MB".format(os.path.getsize(path) / 1000000))

In [ ]:
%%time 
# SQLITE
path = dump_path.format("db")
engine = create_engine('sqlite:///{}'.format(path), echo=True)
df.to_sql("series_tiempo", engine, index=False, if_exists="replace")
print("{}MB".format(os.path.getsize(path) / 1000000))

In [ ]:
%%time 
# ACCESS
# set up some constants
DRV = '{Microsoft Access Driver (*.mdb)}'; PWD = 'pw'

# connect to db
path = dump_path.format("mdb")
con = pyodbc.connect('DRIVER={};DBQ={};PWD={}'.format(DRV, path, PWD))
cur = con.cursor()

df.to_sql("series_tiempo", con, index=False, if_exists="replace")
print("{}MB".format(os.path.getsize(path) / 1000000))

In [ ]:
%%time 
# SQL
# la gracia acá es hacer bien las tablas con sus índices y pensar si MySQL, Postgresql o SQLite
path = dump_path.format("sql")
df.to_sql(path, "data", merge_cells=False, encoding="utf-8", index=False)
print("{}MB".format(os.path.getsize(path) / 1000000))

In [22]:
%%time 
# DTA
# hay que ver cómo manejar el límite de caracteres
path = dump_path.format("dta")
df_stata = df.copy()
for col in df_stata.columns:
    if df_stata[col].dtype.name == "object":
        # trunca campos de texto a 244 caracteres aceptados por STATA
        df_stata[col] = df_stata[col].str[:244]
df_stata.to_stata(path, write_index=False)
print("{}MB".format(os.path.getsize(path) / 1000000))

1097MB
CPU times: user 20.1 s, sys: 5.21 s, total: 25.3 s
Wall time: 25.6 s


In [ ]:
df.head()